In [ ]:
# 라이브러리 임포트
import folium
from folium.plugins import AntPath
from folium import Map, PolyLine
from branca.element import Template, MacroElement
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from haversine import haversine

In [ ]:
# 역정규화 함수
def inverse_transform_preds(preds, scaler):
    # 위도, 경도, SOG, COG 값만 역정규화
    preds_unscaled = preds.copy()  # 예측된 값을 복사

    # 위도, 경도, SOG, COG를 역정규화
    preds_unscaled[:, :4] = scaler.inverse_transform(preds_unscaled[:, :4])  # 역정규화
    return preds_unscaled

In [ ]:
# 예측 경로 시각화 함수
def visualize_route(initial_seq, preds_inverse, dest_lat, dest_lon):
    # 초기 위치
    start = initial_seq[0, 0][:4]
    start = scaler.inverse_transform([start])
    start_lat = start[0][0] 
    start_lon = start[0][1]
    
    # 지도 생성 (출발지와 목적지가 모두 보이도록 설정)
    route_map = folium.Map(location=[start_lat, start_lon], zoom_start=6)

    # 시작점, 목적지 마커 추가
    folium.Marker([start_lat, start_lon], tooltip='Start', icon=folium.Icon(color='green')).add_to(route_map)
    folium.Marker([dest_lat, dest_lon], tooltip='Destination', icon=folium.Icon(color='red')).add_to(route_map)

    # 예측 경로
    route_coords = [[lat, lon] for lat, lon in preds_inverse[::10, :2]]  # 위도, 경도만 사용
    
    # 예측 경로를 PolyLine으로 시각화
    folium.PolyLine(route_coords, color="blue", weight=4, opacity=0.8).add_to(route_map)
    
    # 예측 경로에 애니메이션 효과 추가
    AntPath(route_coords).add_to(route_map)

    return route_map

In [ ]:
# 예측 결과를 역정규화 후 시각화하는 코드
def predict_and_visualize(model, initial_seq, dest_lat, dest_lon, scaler, max_steps=4800, distance_threshold=0.1):
    # MC Dropout 기반 자가 회귀 수행
    preds = predict_autoregressive_mc_dropout(model, initial_seq, dest_lat, dest_lon, scaler, max_steps, distance_threshold, num_mc_samples=10) 
    
    # 경로 역정규화
    preds = preds.squeeze(1)
    preds_inverse = inverse_transform_preds(preds, scaler)
    
    # 예측 경로 시각화
    route_map = visualize_route(initial_seq.numpy(), preds_inverse, dest_lat, dest_lon)
    predictions = preds_inverse
    
    return route_map, predictions

In [ ]:
# 예시: 초기 시퀀스와 목적지 좌표로 예측 및 시각화
route_map, predictions = predict_and_visualize(model, test_input_seq, dest_lat=dest_lat, dest_lon=dest_lon, scaler=scaler)
route_map.save('predicted_route_map_bm.html')

In [ ]:
# 기존 경로 및 예측 경로 대조 Pipline
def plot_routes_on_map(actual_coords, predicted_coords, metrics):
    # 지도 초기화
    m = folium.Map(location=actual_coords[0], zoom_start=8)

    # 실제 경로 (파란색 실선)
    folium.PolyLine(actual_coords, color='blue', weight=5, opacity=0.7, tooltip='Actual Route').add_to(m)

    # 예측 경로 (빨간색 점선)
    folium.PolyLine(predicted_coords, color='red', weight=3, opacity=0.7, tooltip='Predicted Route', dash_array='10').add_to(m)

    # 범례 + 평가 지표 HTML 템플릿
    legend_html = """
    {{% macro html(this, kwargs) %}}
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        z-index: 9999;
        background-color: white;
        border: 2px solid grey;
        border-radius: 5px;
        padding: 10px;
        font-size: 14px;
        box-shadow: 2px 2px 6px rgba(0,0,0,0.3);
        line-height: 1.6;
    ">
      <b>🗺 경로 범례</b><br>
      <i style="background:blue; width:10px; height:10px; display:inline-block;"></i> 실제 경로<br>
      <i style="background:red; width:10px; height:2px; display:inline-block; border-top: 3px dashed red;"></i> 예측 경로
      <hr style="margin:6px 0;">
      <b>📊 경로 평가 지표</b><br>
      📍 평균 거리 (Haversine): {haversine:.3f} km<br>
      ✅ 유사도: {similarity:.2f}%<br>
      📉 RMSE (위/경도): {rmse:.6f}<br>
      📉 MSE  (위/경도): {mse:.6f}
    </div>
    {{% endmacro %}}
    """.format(
        haversine=metrics['haversine_km'],
        similarity=metrics['similarity_percent'],
        rmse=metrics['rmse'],
        mse=metrics['mse']
    )

    macro = MacroElement()
    macro._template = Template(legend_html)
    m.get_root().add_child(macro)

    return m

temp_df = df.iloc[40:]
values = temp_df[['위도', '경도']].values

actual_coords = values
predicted_coords = predictions[:,:2]

def haversine_average_distance(actual, predicted):
    min_len = min(len(actual), len(predicted))
    total_distance = sum(haversine(a, p) for a, p in zip(actual[:min_len], predicted[:min_len]))
    return total_distance / min_len  # km


def coord_rmse_mse(actual, predicted):
    min_len = min(len(actual), len(predicted))
    actual_np = np.array(actual[:min_len])
    pred_np = np.array(predicted[:min_len])

    mse = mean_squared_error(actual_np, pred_np)
    rmse = np.sqrt(mse)
    return rmse, mse


def evaluate_route(actual_coords, predicted_coords):
    # Haversine 평균 거리
    haversine_avg = haversine_average_distance(actual_coords, predicted_coords)

    # Haversine 기반 퍼센트 유사도 (100km 이상은 0%)
    similarity = max(0, 100 * (1 - haversine_avg / 100))

    # 위도/경도 기준 RMSE & MSE
    rmse, mse = coord_rmse_mse(actual_coords, predicted_coords)

    print(f"📍 평균 거리 (Haversine): {haversine_avg:.3f} km")
    print(f"✅ 유사도: {similarity:.2f}%")
    print(f"📉 RMSE (위도+경도 좌표): {rmse:.6f}")
    print(f"📉 MSE  (위도+경도 좌표): {mse:.6f}")

    return {
        'haversine_km': haversine_avg,
        'similarity_percent': similarity,
        'rmse': rmse,
        'mse': mse,
    }
    
metrics = evaluate_route(actual_coords, predicted_coords)
m = plot_routes_on_map(actual_coords, predicted_coords, metrics)
m.save('route_comparison.html')